#  Analyses of UT cells - scvi


# 0. Loading the libraries

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import scanpy as sc
import numpy as np
import scipy as sp
import pandas as pd

%matplotlib inline
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import seaborn as sb
sb.set_context('notebook')
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib import colors
plt.rcParams['figure.figsize']=(8,8) #rescale figures
plt.rcParams.update({'figure.max_open_warning': 0})

from gprofiler import gprofiler

import scprep
from umap import UMAP
import graphtools as gt
import meld
import magic

In [3]:
import rpy2
print(rpy2.__version__)

3.4.2


In [4]:
# Ignore R warning messages
# Note: this can be commented out to get more verbose R output
import rpy2.rinterface_lib.callbacks
import logging
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)

# Automatically convert rpy2 outputs to pandas dataframes
import rpy2.robjects as ro

from rpy2.robjects import pandas2ri
pandas2ri.activate()

from rpy2.robjects import numpy2ri
numpy2ri.activate()

import anndata2ri
anndata2ri.activate()

%reload_ext rpy2.ipython

sc.settings.verbosity = 3
#sc.set_figure_params(dpi=200, dpi_save=300)
sc.logging.print_versions()

import scvi
import random

-----
anndata     0.7.6
scanpy      1.8.1
sinfo       0.3.1
-----
PIL                 8.3.1
anndata             0.7.6
anndata2ri          0.0.0
autoreload          NA
backcall            0.2.0
beta_ufunc          NA
binom_ufunc         NA
brotli              NA
certifi             2022.09.24
cffi                1.14.6
chardet             4.0.0
charset_normalizer  2.0.0
colorama            0.4.4
concurrent          NA
cycler              0.10.0
cython_runtime      NA
dateutil            2.8.2
decorator           4.4.2
defusedxml          0.7.1
deprecated          1.2.12
dunamai             1.6.0
encodings           NA
future              0.18.2
genericpath         NA
get_version         3.5
google              NA
gprofiler           1.0.0
graphtools          1.5.2
h5py                3.3.0
idna                3.2
igraph              0.9.6
ipykernel           6.1.0
ipython_genutils    0.2.0
ipywidgets          7.6.3
jedi                0.18.0
jinja2              3.0.1
joblib             

<a id="Reading"></a>

# 1. Reading in the data

In [5]:
import anndata
adata = anndata.read_h5ad('../../data/data03-scvi/finalall-adata.h5ad')

In [6]:
# Annotate the data sets
print(adata.obs['orig.ident'].value_counts())
print('')
print(adata.obs['nruns'].value_counts())
print('')

UT_0_1_FALSE      2736
FCZ_6_3_FALSE     2483
RAPA_2_2_FALSE    2460
FCZ_2_2_FALSE     2444
CSP_2_1_FALSE     2075
UT_0_3_FALSE      2050
FCZ_3_2_FALSE     1616
FCZ_2_1_FALSE      948
FCZ_3_3_FALSE      908
FCZ_3_1_FALSE      858
UT_0_2_FALSE       276
Name: orig.ident, dtype: int64

3.0    12237
2.0     6617
Name: nruns, dtype: int64



In [7]:
adata = adata[(adata.obs["drug_day"].isin(["UT_0"]))]

In [8]:
print(adata.obs['orig.ident'].value_counts())
print('')

UT_0_1_FALSE    2736
UT_0_3_FALSE    2050
UT_0_2_FALSE     276
Name: orig.ident, dtype: int64



# 2. MAGIC imputation

Count data are first notmalized for library size and square-root transformed. We then aply MAGIC (Markov Affinity-based Graph Imputation of Cells), an algorithm for denoising and imputation of single cells applied to single-cell RNA sequencing data, as described in Van Dijk D et al. (2018), Recovering Gene Interactions from Single-Cell Data Using Data Diffusion, Cell. [https://www.cell.com/cell/abstract/S0092-8674(18)30724-4]

In [9]:
import magic
data_libnorm, libsize = scprep.normalize.library_size_normalize(adata.X, return_library_size=True)
data_sqrt = scprep.transform.sqrt(data_libnorm)
data_magic = magic.MAGIC().fit_transform(data_sqrt)

Calculating MAGIC...
  Running MAGIC on 5062 cells and 5277 genes.
  Calculating graph and diffusion operator...
    Calculating PCA...


/home/vanessa.d/.local/lib/python3.9/site-packages/magic/magic.py:425: UserWarning: Input matrix contains unexpressed genes. Please remove them prior to running MAGIC.
  warnings.warn(


    Calculated PCA in 0.67 seconds.
    Calculating KNN search...
    Calculated KNN search in 2.97 seconds.
    Calculating affinities...
    Calculated affinities in 2.92 seconds.
  Calculated graph and diffusion operator in 6.58 seconds.
  Calculating imputation...
  Calculated imputation in 2.31 seconds.
Calculated MAGIC in 8.99 seconds.


In [10]:
adata.layers["magic"]=data_magic

In [11]:
gene_list = adata.var.index.values
dge = pd.DataFrame(data_magic,columns = gene_list)
#pd.DataFrame(dge).to_csv("../../data/data03-scvi/ut_0-adjLogcounts_magic.csv")

# 3. scVI

In [12]:
adata.layers["counts"] = adata.X.copy() # preserve counts
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
adata.raw = adata # freeze the state in `.raw`

normalizing counts per cell
    finished (0:00:00)


In [13]:
adata
adata.write('ut_processed.h5ad')

In [ ]:
adata = anndata.read('ut_processed.h5ad')


def process_random_columns(adata, sample_size=0.95):
    all_columns = adata.obs_names.tolist()
    
    # leiden_scVI_vectors = {} # list to store the resulting leiden_scVI vectors
          
    random.shuffle(all_columns)
        
    # Select a random sample of the columns
        
    sample_columns = all_columns[:int(sample_size * len(all_columns))]
               
    tmp_adata = adata[(adata.obs_names.isin(sample_columns))].copy()
        
    scvi.data.setup_anndata(
                tmp_adata,
                layer="counts",
                batch_key='batch',
                continuous_covariate_keys=["log_counts"])
            
    model = scvi.model.SCVI(tmp_adata)
            
    model.train()
            
    tmp_adata.layers["scvi_normalized"] = model.get_normalized_expression(
                library_size=10e4
            )
            
    latent = model.get_latent_representation()
            
    tmp_adata.obsm["X_scVI"] = latent
            
    sc.pp.neighbors(tmp_adata, use_rep="X_scVI")
            
    sc.tl.umap(tmp_adata, min_dist=0.3)
            
    sc.tl.leiden(tmp_adata, key_added="leiden_scVI", resolution=0.4)
    
    leiden_scVI_vectors = pd.DataFrame({'cellid': tmp_adata.obs_names.tolist(), 'leiden_scvi': tmp_adata.obs['leiden_scVI']})
            
    return leiden_scVI_vectors


In [ ]:
results = []
for i in range(100):
    result = process_random_columns(adata)
    results.append(result)
    
# Merge the DataFrames
df_merged = results[0]

for df in results[1:]:
    df_merged = df_merged.merge(df, on='cellid', how='outer')

df_merged.to_csv('scvi_ut_iteration.csv', index=False)